In [1]:
import os
import multiprocessing

import tqdm
import pandas as pd
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F


import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree

In [2]:
# device = torch.device("cpu")
device = torch.device("cuda")

In [3]:
# df_train = pd.read_csv(
#     os.path.join("data", "train.csv")
# )
# df_attrs = pd.read_csv(
#     os.path.join("data", "attr.csv")
# )

In [4]:
# node_feature_cols = [
#     "age",
#     "city_id",
#     "sex",
#     "school",
#     "university",
# ]

# edge_feature_cols = [
#     "t",
#     "x2",
#     "x3",
# ]

# adj_col_names = ["u", "v"]

# def prepare_graph_data(ego_id: int):
#     df_edges = df_train.loc[df_train.loc[:, "ego_id"] == ego_id]
#     df_features = df_attrs.loc[df_attrs.loc[:, "ego_id"] == ego_id]

#     unique_u = set(df_edges.loc[:, "u"])
#     unique_v = set(df_edges.loc[:, "v"])
#     unique_nodes_with_attrs = set(df_features.loc[:, "u"])

#     max_node_idx = max(max(unique_u), max(unique_v))
#     fake_nodes = [idx for idx in range(max_node_idx + 1) if idx not in unique_nodes_with_attrs]
#     if len(fake_nodes) > 0:
#         fake_features = [-1 for _ in fake_nodes]
#         fake_node_to_attrs = {
#             "ego_id": ego_id,
#             "u": list(fake_nodes),
#             "age": fake_features,
#             "city_id": fake_features,
#             "sex": fake_features,
#             "school": fake_features,
#             "university": fake_features,
#         }
#         fake_nodes_df = pd.DataFrame(fake_node_to_attrs)
#         fake_nodes_df = pd.concat([df_features, fake_nodes_df]).sort_values(by="u")

#         result = Data(
#             x=torch.tensor(fake_nodes_df.loc[:, node_feature_cols].to_numpy(), dtype=torch.float32),
#             edge_index=torch.tensor(df_edges.loc[:, adj_col_names].to_numpy(), dtype=torch.int64).T,
#             edge_attr=torch.tensor(df_edges.loc[:, edge_feature_cols].fillna(-1).to_numpy(), dtype=torch.float32),
#             y=torch.tensor(df_edges.loc[:, "x1"].to_numpy(), dtype=torch.float32),
#         ).to(device)

#         return result

In [5]:
# graph_dataset = []
# for ego_id in tqdm.tqdm(df_train.loc[:, "ego_id"].unique()):
#     ego_graph = prepare_graph_data(ego_id)
#     if ego_graph:
#         graph_dataset.append(prepare_graph_data(ego_id))

In [6]:
# graph_dataset = torch.load(os.path.join("data", "preprocessed", "train_tensor.pt"))
graph_dataset = torch.load("data/preprocesed/train_tensor.pt")

In [7]:
graph_dataset

[Data(x=[300, 5], edge_index=[2, 1642], edge_attr=[1642, 3], y=[1642]),
 Data(x=[132, 5], edge_index=[2, 665], edge_attr=[665, 3], y=[665]),
 Data(x=[210, 5], edge_index=[2, 2296], edge_attr=[2296, 3], y=[2296]),
 Data(x=[269, 5], edge_index=[2, 1385], edge_attr=[1385, 3], y=[1385]),
 Data(x=[299, 5], edge_index=[2, 2760], edge_attr=[2760, 3], y=[2760]),
 Data(x=[208, 5], edge_index=[2, 1051], edge_attr=[1051, 3], y=[1051]),
 Data(x=[150, 5], edge_index=[2, 734], edge_attr=[734, 3], y=[734]),
 Data(x=[267, 5], edge_index=[2, 2527], edge_attr=[2527, 3], y=[2527]),
 Data(x=[173, 5], edge_index=[2, 854], edge_attr=[854, 3], y=[854]),
 Data(x=[300, 5], edge_index=[2, 3464], edge_attr=[3464, 3], y=[3464]),
 Data(x=[300, 5], edge_index=[2, 1317], edge_attr=[1317, 3], y=[1317]),
 Data(x=[300, 5], edge_index=[2, 1678], edge_attr=[1678, 3], y=[1678]),
 Data(x=[300, 5], edge_index=[2, 1767], edge_attr=[1767, 3], y=[1767]),
 Data(x=[186, 5], edge_index=[2, 1049], edge_attr=[1049, 3], y=[1049]),
 

In [7]:
# torch.save(
#     graph_dataset,
#     os.path.join("data", "preprocessed", "train_tensor.pt"),
# )

In [24]:
for idx, graph in enumerate(graph_dataset):
    tensor = graph.x
    columns_to_delete = [1, 3, 4]

    # Deleting the specified columns
    updated_tensor = torch.cat([tensor[:, :columns_to_delete[0]], tensor[:, columns_to_delete[0]+1:]], dim=1)
    for i in range(1, len(columns_to_delete)):
        updated_tensor = torch.cat([updated_tensor[:, :columns_to_delete[i]-i], updated_tensor[:, columns_to_delete[i]-i+1:]], dim=1)

    # Printing the updated tensor size
    graph.x = updated_tensor
    graph_dataset[idx] = graph

In [25]:
graph_dataset[0]

Data(x=[300, 2], edge_index=[2, 1642], edge_attr=[1642, 3], y=[1642])

In [26]:
loader = DataLoader(graph_dataset, batch_size=512)

In [32]:
train_loader = DataLoader(graph_dataset[:int(len(graph_dataset) * 0.1)], batch_size=512)
val_loader = DataLoader(graph_dataset[int(len(graph_dataset) * 0.5):int(len(graph_dataset) * 1.)], batch_size=512)
# train_loader = DataLoader(graph_dataset[:512], batch_size=32)
# val_loader = DataLoader(graph_dataset[512:768], batch_size=32)

In [33]:
graph_dataset[0].num_node_features

2

In [34]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # self.conv1 = pyg_nn.SAGEConv((-1, -1), hidden_channels)
        # self.conv2 = pyg_nn.SAGEConv((-1, -1), hidden_channels)
        self.conv1 = pyg_nn.GCNConv(graph_dataset[0].num_node_features, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, hidden_channels)
        self.lin1 = torch.nn.Linear(2 * hidden_channels + 3, hidden_channels // 2)
        self.lin2 = torch.nn.Linear(hidden_channels // 2, 1)
        # self.lin1 = torch.nn.Linear(2 * hidden_channels + 3, hidden_channels)
        # self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x)
        x = F.dropout(x, training=self.training, p=0.05)

        idx_u, idx_v = edge_index
        x = torch.cat([x[idx_u], x[idx_v], edge_attr], dim=-1)
        x = self.lin1(x)
        x = F.leaky_relu(x)
        x = F.dropout(x, training=self.training, p=0.05)
        x = self.lin2(x)
        return x.view(-1)

hidden_state = 32
model = Model(hidden_channels=hidden_state).to(device)

In [35]:
local_val_minimum = 1.00228

In [39]:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

initial_lr = 0.05
reset_interval = 5

optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=reset_interval, gamma=0.1)

for epoch in range(1, 1000):
    total_loss = 0
    model.train()
    optimizer.zero_grad()
    for train_batch in train_loader:
        pred = model(train_batch.x, train_batch.edge_index, train_batch.edge_attr)
        target = train_batch.y
        loss = F.mse_loss(pred, target)
        total_loss += loss
        loss.backward()
        optimizer.step()
    # scheduler.step()
    loss = total_loss / len(loader)

    model.eval()
    val_metric = 0
    for val_batch in val_loader:
        with torch.no_grad():
            pred = model(val_batch.x, val_batch.edge_index, val_batch.edge_attr)
            target = val_batch.y

        val_metric += mean_squared_error(pred.cpu(), target.cpu())
    val_metric = val_metric / len(val_loader)

    if val_metric < local_val_minimum:
        model_name = f"h_{hidden_state}_val_metric_{val_metric}.pt"
        torch.save(
            model.state_dict(),
            # os.path.join("models", model_name),
            f"models{model_name}"
        )
        print(f"save model {model_name}")
        local_val_minimum = val_metric
    
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, val metric: {val_metric}')

Epoch: 001, Loss: 17.5255, val metric: 306.0271720239672
Epoch: 002, Loss: 46.6199, val metric: 985.6819002830376
Epoch: 003, Loss: 41.3918, val metric: 266.6650475970769
Epoch: 004, Loss: 34.7216, val metric: 179.84890178098516
Epoch: 005, Loss: 46.2940, val metric: 15.806169606871524


KeyboardInterrupt: 

In [ ]:
pred

tensor([ 1.3909,  1.1429, -0.1203,  ..., -0.1712, -0.5911, -0.4413],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [225]:
len(pred.unique())

1021141

In [224]:
import math 

